In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
np.bool = np.bool_

#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [ ]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [ ]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

In [ ]:
listings = (spark.read
              .option("header","true")
                .option("inferschema", "true")
              .option("escape","\"")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/airbnb/listings/"))

In [ ]:
reviews = (spark.read
              .option("header","true")
                .option("inferschema", "true")
              .option("escape","\"")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/airbnb/reviews/"))

In [ ]:
listings.limit(5).toPandas()

In [ ]:
reviews.limit(5).toPandas()